## Importações e Configurações

In [1]:
import requests
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.io as pio
import re
import time

pio.renderers.default = "notebook_connected"

BASE_URL = "https://www.nic.br"
HEADERS = {"User-Agent": "Mozilla/5.0"}

DATABASE_NAME = "internet_governance_news.db"

print("✅ Ambiente NIC.br pronto")


✅ Ambiente NIC.br pronto


In [2]:
def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela prontos")

create_database()


✅ Banco e tabela prontos


In [3]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


## Scraper

In [4]:
def carregar_indice_json(codigo="todos", arquivo=1):
    url = f"{BASE_URL}/noticias/indice/{codigo}/indice-{arquivo}.json"
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    return r.json()


In [5]:
def scrape_nic_noticias(total_noticias=1150):
    print("🔄 Scraping NIC.br via JSON...")
    start = time.time()
    dados = carregar_indice_json("todos", 1)

    artigos = []

    for noticia in dados[:total_noticias]:
        title = noticia.get("titulo")
        url = noticia.get("permalink")
        date = noticia.get("data")
        resumo = noticia.get("resumo")

        data = {
            "title": title,
            "date": date,
            "author": "NIC.br",
            "url": url,
            "source": "NIC Notícias"
        }

        artigos.append(data)
        insert_article(**data)

    df = pd.DataFrame(artigos)
    print(f"✅ {len(df)} notícias coletadas")
    print(f"⏱️ {time.time() - start:.2f}s")
    display(df.head(10))
    return df

df_nic = scrape_nic_noticias(total_noticias=1150)


🔄 Scraping NIC.br via JSON...
✅ 500 notícias coletadas
⏱️ 4.06s


,title,date,author,url,source
0,Falha em roteadores gera tráfego anômalo e pre...,2025-12-16,NIC.br,/noticia/na-midia/falha-em-roteadores-gera-tra...,NIC Notícias
1,Brasil vs. EUA: o que separa e o que aproxima ...,2025-12-16,NIC.br,/noticia/na-midia/brasil-vs-eua-o-que-separa-e...,NIC Notícias
2,Brasil tem recorde de escolas com Medidor Educ...,2025-12-15,NIC.br,/noticia/na-midia/brasil-tem-recorde-de-escola...,NIC Notícias
3,Cerca de 30 milhões de brasileiros fizeram apo...,2025-12-15,NIC.br,/noticia/na-midia/cerca-de-30-milhoes-de-brasi...,NIC Notícias
4,Conselheira do CGI.br é destaque na série <i>W...,2025-12-15,NIC.br,/noticia/na-midia/conselheira-do-cgi-br-e-dest...,NIC Notícias
5,'Influprofissionais': linguagem das redes rede...,2025-12-12,NIC.br,/noticia/na-midia/influprofissionais-linguagem...,NIC Notícias
6,"<i>Cyberbullying</i>, como se precaver e procu...",2025-12-12,NIC.br,/noticia/na-midia/i-cyberbullying-i-como-se-pr...,NIC Notícias
7,A relação simbiótica entre jornalismo e educaç...,2025-12-12,NIC.br,/noticia/na-midia/a-relacao-simbiotica-entre-j...,NIC Notícias
8,Para além dos bloqueios: queremos mudanças no...,2025-12-11,NIC.br,/noticia/na-midia/para-alem-dos-bloqueios-quer...,NIC Notícias
9,15ª Semana da Infraestrutura da Internet no Br...,2025-12-11,NIC.br,/noticia/na-midia/15-semana-da-infraestrutura-...,NIC Notícias


In [6]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT title, date, author, url, source
        FROM articles
        WHERE source = 'NIC Notícias'
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total NIC no banco: {len(df_db)}")
display(df_db.head(20))


📦 Total NIC no banco: 499


,title,date,author,url,source
0,Falha em roteadores gera tráfego anômalo e pre...,2025-12-16,NIC.br,/noticia/na-midia/falha-em-roteadores-gera-tra...,NIC Notícias
1,Brasil vs. EUA: o que separa e o que aproxima ...,2025-12-16,NIC.br,/noticia/na-midia/brasil-vs-eua-o-que-separa-e...,NIC Notícias
2,Brasil tem recorde de escolas com Medidor Educ...,2025-12-15,NIC.br,/noticia/na-midia/brasil-tem-recorde-de-escola...,NIC Notícias
3,Cerca de 30 milhões de brasileiros fizeram apo...,2025-12-15,NIC.br,/noticia/na-midia/cerca-de-30-milhoes-de-brasi...,NIC Notícias
4,Conselheira do CGI.br é destaque na série <i>W...,2025-12-15,NIC.br,/noticia/na-midia/conselheira-do-cgi-br-e-dest...,NIC Notícias
5,'Influprofissionais': linguagem das redes rede...,2025-12-12,NIC.br,/noticia/na-midia/influprofissionais-linguagem...,NIC Notícias
6,"<i>Cyberbullying</i>, como se precaver e procu...",2025-12-12,NIC.br,/noticia/na-midia/i-cyberbullying-i-como-se-pr...,NIC Notícias
7,A relação simbiótica entre jornalismo e educaç...,2025-12-12,NIC.br,/noticia/na-midia/a-relacao-simbiotica-entre-j...,NIC Notícias
8,Para além dos bloqueios: queremos mudanças no...,2025-12-11,NIC.br,/noticia/na-midia/para-alem-dos-bloqueios-quer...,NIC Notícias
9,15ª Semana da Infraestrutura da Internet no Br...,2025-12-11,NIC.br,/noticia/na-midia/15-semana-da-infraestrutura-...,NIC Notícias


## Filtro + Gráficos

In [7]:
keywords = [
    "internet", "governança", "dados",
    "segurança", "privacidade", "digital", "IA"
]

pattern = "|".join(keywords)

df_filt = df_db[
    df_db["title"].str.contains(pattern, case=False, na=False)
]

print(f"🔎 Filtradas: {len(df_filt)}")


🔎 Filtradas: 395


In [8]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados")
        return

    # --------------------------
    # Top 15 Notícias
    # --------------------------
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="title",
        orientation="h",
        title="Top 15 Notícias – NIC.br"
    )
    fig1.update_layout(
        height=600,
        yaxis={"categoryorder": "array",
               "categoryarray": top15["title"][::-1]}
    )
    fig1.show()

    # --------------------------
    # Distribuição por Fonte
    # --------------------------
    src = df["source"].value_counts().reset_index()
    src.columns = ["source", "count"]

    fig2 = px.pie(
        src,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # --------------------------
    # Nuvem de Palavras
    # --------------------------
    text = " ".join(df["title"].astype(str)).lower()
    words = re.findall(r"\b[a-záàâãéèêíïóôõöúçñ]{4,}\b", text)

    wc = (
        pd.Series(words)
        .value_counts()
        .head(25)
        .reset_index()
    )
    wc.columns = ["palavra", "freq"]

    fig3 = px.treemap(
        wc,
        path=["palavra"],
        values="freq",
        title="Nuvem de Palavras – NIC.br"
    )
    fig3.show()


In [9]:
# %%
plot_charts(df_filt if not df_filt.empty else df_plot)
